Jumbled mess; not for public consumption. Keeping for archival purposes only.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

np.random.seed(0)

In [2]:
# Loading data
df_train = pd.read_csv('../../data/processed/train_users_2.csv')
df_test = pd.read_csv('../../data/processed/test_users.csv')
target = df_train['country_destination'].values
df_train = df_train.drop(['id', 'country_destination'], axis=1)
id_test = df_test['id']
df_test = df_test.drop(['id'], axis=1)

df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train = df_train.fillna(-2)
df_test = df_test.replace([np.inf, -np.inf], np.nan)
df_test = df_test.fillna(-2)

In [3]:
null_train=df_train.columns[df_train.isnull().any()]
print(null_train)

null_test=df_train.columns[df_train.isnull().any()]
print(null_train)

Index([], dtype='object')
Index([], dtype='object')


In [5]:
# Splitting train and test
X_train = df_train.to_numpy()
le = LabelEncoder()
y_train = le.fit_transform(target)
X_test = df_test.to_numpy()

print(X_train)

[[-1.00000000e+00  2.01000000e+03  6.00000000e+00 ... -2.00000000e+00
  -2.00000000e+00 -2.00000000e+00]
 [ 3.80000000e+01  2.01100000e+03  5.00000000e+00 ... -2.00000000e+00
  -2.00000000e+00 -2.00000000e+00]
 [ 5.60000000e+01  2.01000000e+03  9.00000000e+00 ... -2.00000000e+00
  -2.00000000e+00 -2.00000000e+00]
 ...
 [ 3.20000000e+01  2.01400000e+03  6.00000000e+00 ...  3.42982000e+05
   2.01754118e+04  4.23254679e+04]
 [-1.00000000e+00  2.01400000e+03  6.00000000e+00 ...  3.41609000e+05
   4.61633784e+03  8.90996884e+03]
 [-1.00000000e+00  2.01400000e+03  6.00000000e+00 ...  2.75921000e+06
   6.89802500e+04  2.73217316e+05]]


In [6]:
# Trying SMOTE, then GBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier

smote = SMOTE()

X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# smote_gb = GradientBoostingClassifier(n_estimators=40, learning_rate=0.2,
#                                    max_depth=8, subsample=0.8, random_state=0,
#                                    max_features=int(len(df_train.columns) ** 0.5))
# parameters = {'n_estimators': [40, 50, 60, 70, 80, 90, 100]}

# clfsmote = GridSearchCV(smote_gb, parameters, cv=3)

# clfsmote.fit(X_train_res, y_train_res)

# JUMP TO BOTTOM

In [6]:
from xgboost import XGBClassifier

smote_xg = XGBClassifier(random_state=0)

parameters = {'n_estimators': [10, 30, 90, 270],
              'learning_rate': [0.05, 0.1, 0.2],
              'max_depth': [3, 4, 5]}

smote_grid = GridSearchCV(smote_xg, parameters, cv=3)

smote_grid.fit(X_train_res, y_train_res)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 30, 90, 270], 'learning_rate': [0.05, 0.1, 0.2], 'max_depth': [3, 4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [7]:
print(smote_grid.best_estimator_)
print(smote_grid.best_score_)
print(smote_grid.cv_results_)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.2, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=270,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)
0.44787476346857447
{'mean_fit_time': array([  789.9915808 ,  2204.14271442,  6406.64405759, 19140.48946905,
         921.1424365 ,  2692.819369  ,  7977.4251066 , 23691.78840669,
        1097.1732134 ,  3240.16687409,  9589.44090573, 28492.44999647,
         743.70760282,  2171.10937905,  6419.67270756, 19143.96723167,
         914.7092731 ,  2687.05369337,  7963.60860697, 23680.63061841,
        1098.00649762,  3219.8140227 ,  9529.82833624, 28300.13595104,
         751.13120023,  2167.07763815,  6388.66619094, 19150.86069942,
         919.48051476,  2674.56755058,  7896.6413126 , 23613.06027063,
       

In [ ]:
# Print out the CSV and score it on 

In [13]:
# Create the Classifier model and the Hyperparameter Tuner
gb = GradientBoostingClassifier(n_estimators=40, learning_rate=0.2,
                                    max_depth=8, subsample=0.8, random_state=0,
                                    max_features=int(len(df_train.columns) ** 0.5))
parameters = {'n_estimators': [100]}

clf = GridSearchCV(gb, parameters, cv=3)
print('Determining the optimal number of trees for learning rate of 0.2')
clf.fit(X_train, y_train)

Determining the optimal number of trees for learning rate of 0.2


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [14]:
print(clf.best_estimator_)
print(clf.best_score_)
print(clf.cv_results_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.28731184206211263
{'mean_fit_time': array([674.17498239]), 'std_fit_time': array([18.49426059]), 'mean_score_time': array([5.57380939]), 'std_score_time': array([0.5182566]), 'param_n_estimators': masked_array(data=[100],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 100}], 'split0_test_score': array([0.04154369]), 'split1_test_score': array([0.29194249]), 'split2_test_score': array([0.52847665])

In [ ]:
# Time a Single Run

In [12]:
# Cheater Method — Jupyter Only

%timeit gb.fit(X_train, y_train)

KeyboardInterrupt: 

In [51]:
# Real Version

from time import time

initial_t = time()

testgb = GradientBoostingClassifier(n_estimators=40, learning_rate=0.2,
                                    max_depth=8, subsample=0.8, random_state=0,
                                    max_features=int(len(df_train.columns) ** 0.5))

testgb.fit(X_train, y_train)

# Options:

delta_t = time() - initial_t
print(f"It took {delta_t} seconds to train the model.")

KeyboardInterrupt: 

In [20]:
# Create the Classifier model and the Hyperparameter Tuner
parameters2 = {'n_estimators': [10, 15, 20, 25, 30, 35, 40, 45]}

clf2 = GridSearchCV(gb, parameters2, cv=3)
print('Still determining the optimal number of trees for learning rate of 0.2')
clf2.fit(X_train, y_train)

Still determining the optimal number of trees for learning rate of 0.2


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 15, 20, 25, 30, 35, 40, 45]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
print(clf2.best_estimator_)
print(clf2.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.3708251542508585


In [24]:
parameters3 = {'n_estimators': [6, 7, 8, 9, 10, 11, 12, 13, 14]}

clf3 = GridSearchCV(gb, parameters3, cv=3)
print('Still determining the optimal number of trees for learning rate of 0.2')
clf3.fit(X_train, y_train)

Still determining the optimal number of trees for learning rate of 0.2


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [6, 7, 8, 9, 10, 11, 12, 13, 14]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [25]:
print(clf3.best_estimator_)
print(clf3.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=6,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.405329560414334


In [26]:
parameters4 = {'n_estimators': [1, 2, 3, 4, 5, 6]}

clf4 = GridSearchCV(gb, parameters4, cv=3)
print('Still determining the optimal number of trees for learning rate of 0.2')
clf4.fit(X_train, y_train)

Still determining the optimal number of trees for learning rate of 0.2


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [1, 2, 3, 4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [27]:
print(clf4.best_estimator_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=1,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)


In [28]:
print(clf4.best_score_)

0.4540526865650665


In [30]:
parameters5 = {'n_estimators': [100]}

clf5 = GridSearchCV(gb, parameters5, cv=3)
print('Still determining the optimal number of trees for learning rate of 0.2')
clf5.fit(X_train, y_train)

Still determining the optimal number of trees for learning rate of 0.2


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [31]:
parameters6 = {'n_estimators': [10, 30, 90],
               'learning_rate': [0.05, 0.1, 0.15, 0.2],
               'max_depth': [5, 6, 7, 8]}

clf6 = GridSearchCV(gb, parameters6, cv=3)
print('Running this overnight to see what happens...')
clf6.fit(X_train, y_train)

Running this overnight to see what happens...


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 30, 90], 'learning_rate': [0.05, 0.1, 0.15, 0.2], 'max_depth': [5, 6, 7, 8]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [34]:
print(clf5.best_estimator_)
print(clf5.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.28731184206211263


In [35]:
print(clf6.best_estimator_)
print(clf6.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=6,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.5434361984717804


In [52]:
parameters7 = {'n_estimators': [10, 30, 90],
               'learning_rate': [0.05, 0.1, 0.15, 0.2],
               'max_depth': [6],
               'max_features': [int(len(df_train.columns) ** 0.5),
                                int(len(df_train.columns) / 3),
                                int(len(df_train.columns) / 2)]}

clf7 = GridSearchCV(gb, parameters7, cv=3)
print('Running this overnight to see what happens...')
clf7.fit(X_train, y_train)

Running this overnight to see what happens...


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.2, loss='deviance', max_depth=8,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples...      subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [10, 30, 90], 'learning_rate': [0.05, 0.1, 0.15, 0.2], 'max_depth': [6], 'max_features': [13, 56, 85]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [53]:
print(clf7.best_estimator_)
print(clf7.best_score_)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.05, loss='deviance', max_depth=6,
              max_features=13, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10,
              n_iter_no_change=None, presort='auto', random_state=0,
              subsample=0.8, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)
0.5434361984717804


In [41]:
le = LabelEncoder()
y_test = le.fit_transform(id_test)

clf7.score(X_test, y_test)

1.6104096882246844e-05

In [42]:
clf.score(X_test, y_test)

0.0

In [43]:
clf2.score(X_test, y_test)

0.0

In [44]:
clf3.score(X_test, y_test)

0.0

In [45]:
clf4.score(X_test, y_test)

1.6104096882246844e-05

In [46]:
clf5.score(X_test, y_test)

0.0

In [8]:
y_pred = smote_grid.predict_proba(X_test)

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

In [9]:
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('sub_smote.csv',index=False)

In [ ]:
parameters8 = {'n_estimators': [500, 1000],
               'learning_rate': [0.05],
               'max_depth': [8],
               'max_features': [int(len(df_train.columns) ** 0.5),
                                int(len(df_train.columns) / 3),
                                int(len(df_train.columns) / 2)]}

clf8 = GridSearchCV(gb, parameters8, cv=5)

clf8.fit(X_train, y_train)

In [ ]:
print("poop")

In [16]:
y_pred = clfsmote.predict_proba(X_test)

#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(id_test)):
    idx = id_test[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

In [17]:
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('subsmote.csv',index=False)

In [7]:
# Adaboost SMOTE
from sklearn.ensemble import AdaBoostClassifier

smote_gb = AdaBoostClassifier()
parameters = {'n_estimators': [40, 50, 60, 70, 80, 90, 100]}

clfsmote2 = GridSearchCV(smote_gb, parameters, cv=3)

clfsmote2.fit(X_train_res, y_train_res)

GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [40, 50, 60, 70, 80, 90, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
print(clfsmote2.best_estimator_)
print(clfsmote2.best_score_)
print(clfsmote2.cv_results_)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=100, random_state=None)
0.1539127048489277
{'mean_fit_time': array([ 277.45087488,  327.68592842,  466.76354225,  602.69548122,
        596.75344912,  814.52127306, 8273.60351125]), 'std_fit_time': array([8.66353348e+00, 4.98014633e+00, 6.82749176e+01, 5.62546461e+01,
       7.19586211e+01, 2.38505865e+01, 1.03885392e+04]), 'mean_score_time': array([ 8.27778848,  9.93573236, 14.52122498, 17.95810596, 17.94151123,
       24.46052329, 26.02718544]), 'std_score_time': array([0.63986763, 0.2332119 , 2.09516008, 1.2687219 , 1.26027955,
       0.18229131, 4.93457478]), 'param_n_estimators': masked_array(data=[40, 50, 60, 70, 80, 90, 100],
             mask=[False, False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_estimators': 40}, {'n_estimators': 50}, {'n_estimators': 60}, {'n_estimators': 70}, {'n_estimators': 80}, {'n_estimators':